# Spark Streaming com Twitter

In [4]:
!pip install requests_oauthlib

    100% |████████████████████████████████| 133kB 4.8MB/s 
  Running setup.py bdist_wheel for oauthlib ... - \ done
  Stored in directory: /gpfs/fs01/user/sf2f-8f4bf40bb9b7e1-6bd8badf036a/.cache/pip/wheels/84/98/7a/fba7268f61097bea6081cbe5480bc439b38975748ea7684fd5
Successfully built oauthlib


In [10]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime,strftime
import requests
import time
import string
import ast
import json

In [17]:
#Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [18]:
#Frequencia de update em segundos
INTERVALO_BATCH = 5

In [19]:
#Criando o Streaming Context
scc = StreamingContext(sc,INTERVALO_BATCH)

In [20]:
#Removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line : line != header)


NameError: name 'arquivo' is not defined

In [ ]:
type(dataset)

In [21]:
# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(line):
  row = line.split(',')
  sentimento = row[1]
  tweet = row[3].strip()
  translator = str.maketrans({key: None for key in string.punctuation})
#translator = re.compile('[%s]' % re.escape(string.punctuation))
#tweet = regex.sub('', tweet)
  tweet = tweet.translate(translator)
  tweet = tweet.split(' ')
  tweet_lower = []
  for word in tweet:
    tweet_lower.append(word.lower())
  return (tweet_lower, sentimento)

SyntaxError: unexpected EOF while parsing (<ipython-input-21-1bf5fdee032f>, line 2)

In [ ]:
# Aplcia a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
# Cria um objeto SentimentAnalyzer 
sentiment_analyzer = SentimentAnalyzer()